In [31]:
# reference: https://qiita.com/Broccolingual/items/aa1f48454b9972b82d63
# check whether I can scrape from GoogleMap
import urllib.robotparser

# read robots.txt
robots_txt_url = 'https://google.com/robots.txt'
rp = urllib.robotparser.RobotFileParser()
rp.set_url(robots_txt_url)
rp.read()

# search whether I can scrape from the website
user_agent = '*'
url = 'https://www.google.com/maps'
result = rp.can_fetch(user_agent, url)
if result:
    print('I can scrape from this webstie.')
else:
    print('I cannot scrape from this webstie.')
print("website: {}".format(url))

I cannot scrape from this webstie.
website: https://www.google.com/maps


In [32]:
# reference: https://qiita.com/Broccolingual/items/aa1f48454b9972b82d63
# check whether I can scrape from MapFan

import urllib.robotparser

# read robots.txt
robots_txt_url = 'https://mapfan.com/robots.txt'
rp = urllib.robotparser.RobotFileParser()
rp.set_url(robots_txt_url)
rp.read()

# search whether I can scrape from the website
user_agent = '*'
url = 'https://mapfan.com/map'
result = rp.can_fetch(user_agent, url)
if result:
    print('I can scrape from this webstie.')
else:
    print('I cannot scrape from this webstie.')
print("website: {}".format(url))

I can scrape from this webstie.
website: https://mapfan.com/map


In [33]:
# import the modules and libraries
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import re

In [34]:
# https://chromedriver.chromium.org/downloads
# version 103r"C:\Users\81808\document1\chromedriver.exe"
# define the driver
# please specify your driver
driver = webdriver.Chrome(r"C:\Users\81808\document1\chromedriver.exe")
sleep(2)

In [35]:
# define all prefectures in Japan
# Japanese words mean each prefecture in Japan like Tokyo, Kyoto
prefectures = ['北海道',
    '青森県',
    '岩手県',
    '宮城県',
    '秋田県',
    '山形県',
    '福島県',
    '茨城県',
    '栃木県',
    '群馬県',
    '埼玉県',
    '千葉県',
    '東京都',
    '神奈川県',
    '新潟県',
    '富山県',
    '石川県',
    '福井県',
    '山梨県',
    '長野県',
    '岐阜県',
    '静岡県',
    '愛知県',
    '三重県',
    '滋賀県',
    '京都府',
    '大阪府',
    '兵庫県',
    '奈良県',
    '和歌山県',
    '鳥取県',
    '島根県',
    '岡山県',
    '広島県',
    '山口県',
    '徳島県',
    '香川県',
    '愛媛県',
    '高知県',
    '福岡県',
    '佐賀県',
    '長崎県',
    '熊本県',
    '大分県',
    '宮崎県',
    '鹿児島県',
    '沖縄県']

In [64]:
# scrape the name, latitude, and longitude of IKEA in MapFan

# define the search word and output list
search_word = "IKEA"
lists_IKEA = []
name_list = []

# loop for each prefecture
for prefecture in prefectures:
    
    # go to the MapFan site
    driver.get('https://mapfan.com/map/spots/search?c=35.681195934673795,139.7672311841094,12&s=std,pc,ja&p=none')
    sleep(1)        

    # search the search bar
    # enter the search word
    # get the result of the search
    search_input = driver.find_element_by_xpath('/html/body/mf-root/div/mat-sidenav-container/mat-sidenav-content/mf-map-page-map/mat-sidenav-container/mat-sidenav-content/div/div[1]/mf-map-page-search/div/form/mat-toolbar/mat-toolbar-row/mf-suggest-field/mat-form-field/div/div[1]/div[3]/input')
    search_input.send_keys("{} {}".format(search_word, prefecture))
    search_input.send_keys(Keys.ENTER)


    # loop for name list
    for elemh4 in driver.find_elements_by_xpath('//h4'):
        elem = elemh4.find_element_by_xpath("../../..")
        
        # click one of the componenets in the list
        elem.click()
        sleep(1)
        
        # get the url of the component
        s = driver.current_url
        list = []
        
        # search latitude and longitude by regular expression
        match = re.search(r'c=(\d+.\d+),(\d+.\d+)', s)
        
        # get the name of the component
        name = elemh4.text
        
        # substitute for the weird words
        table = str.maketrans({
            '\u3000': ' ',
            ' ': '',
            '\t': ''
        })
        name = name.translate(table)
        
        # full-width to half-width
        name = name.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
        
        # get latitude and longitude
        try:
            latitude = match.group(1)
            longitude = match.group(2)
        except: 
            pass
        
        # if there are same names of the components, delete one of such data
        if not name in name_list:
            name_list.append(name)
            
            # find the IKEA by regular expression
            if re.compile(r"^IKEA.+?").search(name) and not re.compile(r".+?駐車場").search(name):
                
                # append the name, latitude, and longitude
                name2 = name.replace("IKEA", "IKEA ", 1)
                list.append(name2)
                list.append(latitude)
                list.append(longitude)
                lists_IKEA.append(list)

In [65]:
# show the lists of IKEA
for i in range(len(lists_IKEA)):
    print(lists_IKEA[i])

['IKEA 仙台', '38.225443683680965', '140.88619132558006']
['IKEA 新三郷', '35.85624087928599', '139.87055303188004']
['IKEA  Tokyo-Bay', '35.681556817882', '139.99198855709005']
['IKEA 立川', '35.706808311052995', '139.41189273079']
['IKEA 渋谷', '35.660404695921', '139.69962571292']
['IKEA 新宿', '35.69008482610501', '139.70483532992']
['IKEA 原宿', '35.67064963963999', '139.70326886768999']
['IKEA 港北', '35.522646780966994', '139.59003351663']
['IKEA 長久手', '35.1762716561', '137.07542118986999']
['IKEA 鶴浜', '34.63688919706101', '135.44964575230006']
['IKEA 神戸', '34.66434710284399', '135.21861488816']
['IKEA 福岡新宮', '33.71003664893698', '130.44986835966006']


In [40]:
# scrape the name, latitude, and longitude of COSTCO in MapFan

# define the search word and output list
search_word = "コストコ" # COSTCO

lists_COSTCO = []
name_list = []

# loop for each prefecture
for prefecture in prefectures:
    
    # go to the MapFan site
    driver.get('https://mapfan.com/map/spots/search?c=35.681195934673795,139.7672311841094,12&s=std,pc,ja&p=none')
    sleep(1)        

    # search the search bar
    # enter the search word
    # get the result of the search
    search_input = driver.find_element_by_xpath('/html/body/mf-root/div/mat-sidenav-container/mat-sidenav-content/mf-map-page-map/mat-sidenav-container/mat-sidenav-content/div/div[1]/mf-map-page-search/div/form/mat-toolbar/mat-toolbar-row/mf-suggest-field/mat-form-field/div/div[1]/div[3]/input')
    search_input.send_keys("{} {}".format(search_word, prefecture))
    search_input.send_keys(Keys.ENTER)

    # loop for name list
    count = 1
    for elemh4 in driver.find_elements_by_xpath('//h4'):
        elem = elemh4.find_element_by_xpath("../../..")
        
        # use p_text to get rid of the uninteded words
        p_text = ""
        if count < len(driver.find_elements_by_xpath('//h4')):
            elem_aboves = elemh4.find_elements_by_xpath("../p")
            try: 
                p_text = elem_aboves[1].text
                count += 1
            except:
                pass
        
        # click one of the componenets in the list
        elem.click()
        sleep(1)
        
        # get the url of the component
        s = driver.current_url
        list = []
        
        # search latitude and longitude by regular expression
        match = re.search(r'c=(\d+.\d+),(\d+.\d+)', s)
        
        # get the name of the component
        name = elemh4.text
        
        # substitute for the weird words
        table = str.maketrans({
            '\u3000': ' ',
            ' ': '',
            '\t': ''
        })
        name = name.translate(table)
        
        # full-width to half-width
        name = name.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
        
        # get latitude and longitude
        try:
            latitude = match.group(1)
            longitude = match.group(2)
        except:
            pass
        
        # if there are same names of the components, delete one of such data
        if not name in name_list:
            name_list.append(name)
            
            # find the COSTCO by regular expression
            if p_text == "[コストコ]":
                if re.compile(r"^コストコホールセール.+?").search(name) and not re.compile(r".+?駐車場").search(name):
                    
                    # append the name, latitude, and longitude
                    name2 = name.replace("コストコホールセール", "COSTCO WHOLESALE ", 1)
                    list.append(name2)
                    list.append(latitude)
                    list.append(longitude)
                    lists_COSTCO.append(list)

In [ ]:
# show the lists of COSTCO
for i in range(len(lists_COSTCO)):
    print(lists_COSTCO[i])

['COSTCO WHOLESALE 札幌倉庫店', '42.97714958084899', '141.46613095222006']
['COSTCO WHOLESALE 石狩倉庫店', '43.18288071521701', '141.30995367923003']
['COSTCO WHOLESALE 富谷倉庫店', '38.393205513078016', '140.87690385329006']
['COSTCO WHOLESALE かみのやま倉庫店', '38.184597840881', '140.2969830723']
['COSTCO WHOLESALE つくば倉庫店', '36.09601115091199', '140.07982765534007']
['COSTCO WHOLESALE ひたちなか倉庫店', '36.39984808167401', '140.58650123243']
['COSTCO WHOLESALE 前橋倉庫店', '36.33266671395701', '139.09394315829002']
['COSTCO WHOLESALE 入間倉庫店', '35.81121548497501', '139.37915348067997']
['COSTCO WHOLESALE 新三郷倉庫店', '35.861191314663984', '139.86412975177996']
['COSTCO WHOLESALE 幕張倉庫店', '35.655062261211', '140.02785784055004']
['COSTCO WHOLESALE 木更津倉庫店', '35.427672799652', '139.91656937685002']
['COSTCO WHOLESALE 千葉ニュータウン倉庫店', '35.806365227040985', '140.12893913669996']
['COSTCO WHOLESALE 多摩境倉庫店', '35.60110888390899', '139.37672830395002']
['COSTCO WHOLESALE 座間倉庫店', '35.48240832811399', '139.42702058844998']
['COSTCO WHOLE

In [41]:
# scrape the name, latitude, and longitude of LALAPORT in MapFan

# define the search word and output list
search_word = "ララポート" # LALAPORT

lists_LALAPORT = []
name_list = []
   
# go to the MapFan site
driver.get('https://mapfan.com/map/spots/search?c=35.681195934673795,139.7672311841094,12&s=std,pc,ja&p=none')
sleep(1)        

# search the search bar
# enter the search word
# get the result of the search
search_input = driver.find_element_by_xpath('/html/body/mf-root/div/mat-sidenav-container/mat-sidenav-content/mf-map-page-map/mat-sidenav-container/mat-sidenav-content/div/div[1]/mf-map-page-search/div/form/mat-toolbar/mat-toolbar-row/mf-suggest-field/mat-form-field/div/div[1]/div[3]/input')
search_input.send_keys("{}".format(search_word))
search_input.send_keys(Keys.ENTER)


count = 1
count_same_LALAPORT = 1

# loop for name list
for elemh4 in driver.find_elements_by_xpath('//h4'):
    elem = elemh4.find_element_by_xpath("../../..")
    
    # use p_text to get rid of the unintended words
    p_text = ""
    if count < len(driver.find_elements_by_xpath('//h4')):
        elem_aboves = elemh4.find_elements_by_xpath("../p")
        try: 
            p_text = elem_aboves[1].text
            count += 1
        except:
            pass
    
    # click one of the componenets in the list
    elem.click()
    sleep(1)
    
    # get the url of the component
    s = driver.current_url
    list = []
    
    # search latitude and longitude by regular expression
    match = re.search(r'c=(\d+.\d+),(\d+.\d+)', s)
    
    # get the name of the component
    name = elemh4.text
    
    # substitute for the weird words
    table = str.maketrans({
        '\u3000': ' ',
        ' ': '',
        '\t': ''
    })
    name = name.translate(table)
    
    # full-width to half-width
    name = name.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
    
    # get latitude and longitude
    try:
        latitude = match.group(1)
        longitude = match.group(2)
    except:
        pass
    
    # find the LALAPORT by regular expression
    if re.compile(r".*ららぽーと.*").search(name) or re.compile(r".*ララポート.*").search(name):
        
        # if there are same names of the components, change the name of such data
        if name in name_list:
            count_same_LALAPORT += 1
            name += str(count_same_LALAPORT)
            
        name_list.append(name)
        name2 = name.replace("ララポート", "LALAPORT ", 1)
        name2 = name2.replace("ららぽーと", "LALAPORT ", 1)
        
        # append the name, latitude, and longitude
        list.append(name2)
        list.append(latitude)
        list.append(longitude)
        lists_LALAPORT.append(list)

In [42]:

# show the lists of LALAPORT
for i in range(len(lists_LALAPORT)):
    print(lists_LALAPORT[i])

['LALAPORT ', '38.43063594238902', '141.23910485250997']
['LALAPORT 2', '35.44699294463001', '133.05749589384004']
['LALAPORT ', '43.19981920856199', '141.76535123662006']
['LALAPORT 3', '35.97458036929501', '139.33877577155']
['LALAPORT 4', '34.83401150959', '138.32616539002004']
['LALAPORT 5', '34.83401150959', '138.32616539002004']
['LALAPORT 6', '32.928177295954995', '132.71087850892002']
['LALAPORT 7', '32.928177295954995', '132.71087850892002']
['LALAPORT 8', '37.71445611826499', '139.00252402801993']
['LALAPORT A', '35.396270099555', '136.942020758']
['LALAPORT A', '35.35465148442401', '132.74589642507']
['LALAPORT C', '35.35442097222901', '132.74626451619997']
['LALAPORT B', '35.396153931707985', '136.94196402501007']
['LALAPORT 加治木', '31.733010324073007', '130.66783180480002']
['LALAPORT 蓮潟', '37.46343538401399', '138.82178920262004']
['LALAPORT 藤山', '36.12597809628801', '140.08722091107006']
['LALAPORT 松崎', '31.591358758221006', '130.55854479616005']
['リフォームショップLALAPORT ', '4

In [58]:
# scrape the name, latitude, and longitude of NITORI in MapFan

# define the search word and output list
search_word = "ニトリ" # NITORI

lists_NITORI = []
name_list = []
   
# go to the MapFan site
driver.get('https://mapfan.com/map/spots/search?c=35.681195934673795,139.7672311841094,12&s=std,pc,ja&p=none')
sleep(1)        


# search the search bar
# enter the search word
# get the result of the search
search_input = driver.find_element_by_xpath('/html/body/mf-root/div/mat-sidenav-container/mat-sidenav-content/mf-map-page-map/mat-sidenav-container/mat-sidenav-content/div/div[1]/mf-map-page-search/div/form/mat-toolbar/mat-toolbar-row/mf-suggest-field/mat-form-field/div/div[1]/div[3]/input')
search_input.send_keys("{}".format(search_word))
search_input.send_keys(Keys.ENTER)


count = 1
count_same_NITORI = 1

# loop for name list
for elemh4 in driver.find_elements_by_xpath('//h4'):
    elem = elemh4.find_element_by_xpath("../../..")
    
    # use p_text to get rid of unintended words
    p_text = ""
    if count < len(driver.find_elements_by_xpath('//h4')):
        elem_aboves = elemh4.find_elements_by_xpath("../p")
        try: 
            p_text = elem_aboves[1].text
            count += 1
        except:
            pass
    
    # click one of the componenets in the list
    elem.click()
    sleep(1)
    
    # get the url of the component
    s = driver.current_url
    list = []
    
    # search latitude and longitude by regular expression
    match = re.search(r'c=(\d+.\d+),(\d+.\d+)', s)
    
    # get the name of the component
    name = elemh4.text
    
    # substitute for the weird words
    table = str.maketrans({
        '\u3000': ' ',
        ' ': '',
        '\t': ''
    })
    name = name.translate(table)
    
    # full-width to half-width
    name = name.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
    
    # get latitude and longitude
    try:
        latitude = match.group(1)
        longitude = match.group(2)
    except:
        pass
    
    # find the NITORI by regular expression
    if re.compile(r".*ニトリ.*").search(name) or re.compile(r".*にとり.*").search(name):
        
        # if there are same names of the components, change the name of such data
        if name in name_list:
            count_same_NITORI += 1
            name += str(count_same_NITORI)
            
        name_list.append(name)
        name2 = name.replace("ニトリ", "NITORI ", 1)
        name2 = name2.replace("にとり", "NITORI ", 1)
        
        # append the name, latitude, and longitude
        list.append(name2)
        list.append(latitude)
        list.append(longitude)
        lists_NITORI.append(list)

In [59]:
# show the lists of NITORI
for i in range(len(lists_NITORI)):
    print(lists_NITORI[i])

['NITORI ', '35.77417954708902', '139.34164045544003']
['NITORI 2', '34.61679173639598', '135.47337757334003']
['NITORI  鹿嶋店', '34.61679173639598', '135.47337757334003']
['NITORI  安城店', '34.981576708158016', '137.09845148960994']
['NITORI  一宮店', '35.314800252954', '136.81713253144994']
['NITORI  滑川店', '36.755973370377006', '137.36266787501995']
['NITORI  南砂店', '35.67204604126199', '139.8269964295']
['NITORI 鹿嶋店', '35.93206604298899', '140.63982715168004']
['NITORI 佐伯店', '32.943266505181015', '131.90000916435997']
['NITORI 安城店', '34.98157112550901', '137.09845361461998']
['NITORI 石巻店', '38.44854829293399', '141.27055571943004']
['NITORI 一関店', '38.93822377183898', '141.11691299545998']
['NITORI 一宮店', '35.31480891847501', '136.81712603171002']
['NITORI 今市店', '36.729286283354', '139.70631352889995']
['NITORI 柏崎店', '37.387658545782', '138.58836387120004']
['NITORI 郡山店', '37.376063575744', '140.37512938689997']
['NITORI 沼南店', '35.831921250112', '140.01127981935997']
['NITORI 新庄店', '38.765409

In [45]:
# scrape the name, latitude, and longitude of DECOHOME in MapFan

# define the search word and output list
search_word = "デコホーム" # DECOHOME

lists_DECOHOME = []
name_list = []
   
# go to the MapFan site
driver.get('https://mapfan.com/map/spots/search?c=35.681195934673795,139.7672311841094,12&s=std,pc,ja&p=none')
sleep(1)        


# search the search bar
# enter the search word
# get the result of the search
search_input = driver.find_element_by_xpath('/html/body/mf-root/div/mat-sidenav-container/mat-sidenav-content/mf-map-page-map/mat-sidenav-container/mat-sidenav-content/div/div[1]/mf-map-page-search/div/form/mat-toolbar/mat-toolbar-row/mf-suggest-field/mat-form-field/div/div[1]/div[3]/input')
search_input.send_keys("{}".format(search_word))
search_input.send_keys(Keys.ENTER)


count = 1
count_same_NITORI = 1

# loop for name list
for elemh4 in driver.find_elements_by_xpath('//h4'):
    elem = elemh4.find_element_by_xpath("../../..")
    
    # use p_text to get rid of unintended words
    p_text = ""
    if count < len(driver.find_elements_by_xpath('//h4')):
        elem_aboves = elemh4.find_elements_by_xpath("../p")
        try: 
            p_text = elem_aboves[1].text
            count += 1
        except:
            pass
    
    # click one of the componenets in the list
    elem.click()
    sleep(1)
    
    # get the url of the component
    s = driver.current_url
    list = []
    
    # search latitude and longitude by regular expression
    match = re.search(r'c=(\d+.\d+),(\d+.\d+)', s)
    
    # get the name of the component
    name = elemh4.text
    
    # substitute for the weird words
    table = str.maketrans({
        '\u3000': ' ',
        ' ': '',
        '\t': ''
    })
    name = name.translate(table)
    
    # full-width to half-width
    name = name.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
    
    # get latitude and longitude
    try:
        latitude = match.group(1)
        longitude = match.group(2)
    except:
        pass
    
    # find the DECOHOME by regular expression
    if re.compile(r".*デコホーム.*").search(name):
        
        # if there are same names of the components, change the name of such data
        if name in name_list:
            count_same_NITORI += 1
            name += str(count_same_NITORI)
            
        name_list.append(name)
        name2 = name.replace("デコホーム", "DECO HOME ", 1)
        
        # append the name, latitude, and longitude
        list.append(name2)
        list.append(latitude)
        list.append(longitude)
        lists_DECOHOME.append(list)

In [46]:
# show the lists of station
for i in range(len(lists_DECOHOME)):
    print(lists_DECOHOME[i])

['DECO HOME ', '35.05282584346601', '135.73381804584005']
['DECO HOME 2', '35.05282584346601', '135.73381804584005']
['DECO HOME LICOPA鶴見店', '35.508958373008994', '139.68619127334']
['DECO HOME Li-Ka1920店', '31.582867920690973', '130.54297610096']
['DECO HOME 浦和パルコ店', '35.85862426554699', '139.65917542764998']
['DECO HOME 笹塚駅前店', '35.67383636774298', '139.66905727294']
['DECO HOME 静岡パルコ店', '34.972740388012966', '138.38565972023002']
['DECO HOME 西友浦安店', '35.66577472253999', '139.89447895777994']
['DECO HOME 西友川西店', '34.828541851126005', '135.41228154898']
['DECO HOME 西友行徳店', '35.68404681005299', '139.91531745426005']
['DECO HOME 西友与野店', '35.884804409998', '139.63183944953994']
['DECO HOME 二俣川駅前店', '35.46484690868', '139.53197873606996']
['DECO HOME COCOSA熊本下通店', '32.80224965918002', '130.70883439226998']
['DECO HOME 北上尾PAPA店', '35.987747039426', '139.57963273468']
['DECO HOME nonowa武蔵小金井店', '35.701136841031996', '139.50506838833996']
['DECO HOME 尼崎つかしん店', '34.76109528592998', '135.41950

In [60]:
# scrape the name, latitude, and longitude of station in MapFan

# define the station list
# the name of the station are like Tokyo, Nagoya, Osaka, etc
stations = ['札幌駅', '仙台駅', '福井駅', '富山駅', '東京駅', '名古屋駅', '大阪駅', '神戸駅 JR', '広島駅', '博多駅']

# define the search word and output list
lists_station = []
name_list = []

# loop for each stations
for station in stations:
    
    # go to the MapFan site
    driver.get('https://mapfan.com/map/spots/search?c=35.681195934673795,139.7672311841094,12&s=std,pc,ja&p=none')
    sleep(1)        

    # search the search bar
    # enter the search word
    # get the result of the search
    search_input = driver.find_element_by_xpath('/html/body/mf-root/div/mat-sidenav-container/mat-sidenav-content/mf-map-page-map/mat-sidenav-container/mat-sidenav-content/div/div[1]/mf-map-page-search/div/form/mat-toolbar/mat-toolbar-row/mf-suggest-field/mat-form-field/div/div[1]/div[3]/input')
    search_input.send_keys("{}".format(station))
    search_input.send_keys(Keys.ENTER)

    
    count = 1
    elemh4 = driver.find_element_by_xpath('//h4')
    elem = elemh4.find_element_by_xpath("../../..")
    
    # use p_text to get rid of unintended words
    p_text = ""
    if count < len(driver.find_elements_by_xpath('//h4')):
        elem_aboves = elemh4.find_elements_by_xpath("../p")
        try: 
            p_text = elem_aboves[1].text
            count += 1
        except:
            pass
    
    # click one of the componenets in the list
    elem.click()
    sleep(1)
    
    # get the url of the component
    s = driver.current_url
    list = []
    
    # search latitude and longitude by regular expression
    match = re.search(r'c=(\d+.\d+),(\d+.\d+)', s)
    
    # get the name of the component
    name = elemh4.text
    
    # substitute for the weird words
    table = str.maketrans({
        '\u3000': ' ',
        ' ': '',
        '\t': ''
    })
    name = name.translate(table)
    
    # full-width to half-width
    name = name.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
    
    # get latitude and longitude
    try:
        latitude = match.group(1)
        longitude = match.group(2)
    except:
        pass
    
    if re.compile(r".*駅.*").search(name):
        # if there are same names of the components, delete one of such data
        if not name in name_list:
            
            # append the name, latitude, and longitude
            name_list.append(name)
            list.append(name)
            list.append(latitude)
            list.append(longitude)
            lists_station.append(list)

In [61]:
# show the lists of station
for i in range(len(lists_station)):
    print(lists_station[i])

['札幌駅(函館本線)', '43.068563958132046', '141.35071384334003']
['仙台駅(南北線)', '38.259720291690996', '140.87997050533']
['福井駅(北陸本線)', '36.06212686341699', '136.22330723367998']
['富山駅(高山本線)', '36.701729399517006', '137.21322711200003']
['東京駅(京葉線)', '35.677612425672024', '139.76513708225002']
['名古屋駅(関西本線)', '35.17064305915302', '136.88169450140003']
['大阪駅(JR宝塚線)', '34.70228873074899', '135.49535090414997']
['神戸駅(JR神戸線)', '34.679236567185', '135.17824156888003']
['広島駅(芸備線)', '34.39761979445599', '132.47536305110998']
['博多駅(空港線)', '33.590188038707', '130.42068498689002']


In [49]:
# define the list of the areas like Tokyo, Osaka, etc
areas = ['北海道',
    '青森',
    '岩手',
    '宮城',
    '秋田',
    '山形',
    '福島',
    '茨城',
    '栃木',
    '群馬',
    '埼玉',
    '千葉',
    '東京',
    '神奈川',
    '新潟',
    '富山',
    '石川',
    '福井',
    '山梨',
    '長野',
    '岐阜',
    '静岡',
    '愛知',
    '三重',
    '滋賀',
    '京都',
    '大阪',
    '兵庫',
    '奈良',
    '和歌山',
    '鳥取',
    '島根',
    '岡山',
    '広島',
    '山口',
    '徳島',
    '香川',
    '愛媛',
    '高知',
    '福岡',
    '佐賀',
    '長崎',
    '熊本',
    '大分',
    '宮崎',
    '鹿児島',
    '沖縄']

In [50]:
# scrape the name, latitude, and longitude of all stations in MapFan

# define the search word and output list
search_word = "駅"

lists_all_stations = []
name_list = []

# loop for each area
for area in areas:
    # go to the MapFan site
    driver.get('https://mapfan.com/map/spots/search?c=35.681195934673795,139.7672311841094,12&s=std,pc,ja&p=none')
    sleep(1)        

    # search the search bar
    # enter the search word
    # get the result of the search
    search_input = driver.find_element_by_xpath('/html/body/mf-root/div/mat-sidenav-container/mat-sidenav-content/mf-map-page-map/mat-sidenav-container/mat-sidenav-content/div/div[1]/mf-map-page-search/div/form/mat-toolbar/mat-toolbar-row/mf-suggest-field/mat-form-field/div/div[1]/div[3]/input')
    search_input.send_keys("{} {}".format(search_word, area))
    search_input.send_keys(Keys.ENTER)


    count = 1
    
    # loop for name list
    for elemh4 in driver.find_elements_by_xpath('//h4'):
        elem = elemh4.find_element_by_xpath("../../..")
        
        #use p_text to get rid of the unintended words
        p_text = ""
        if count < len(driver.find_elements_by_xpath('//h4')):
            elem_aboves = elemh4.find_elements_by_xpath("../p")
            try: 
                p_text = elem_aboves[1].text
                count += 1
            except:
                pass
        
        # click one of the componenets in the list
        elem.click()
        sleep(1)
        
        # get the url of the component
        s = driver.current_url
        list = []
        
        # search latitude and longitude by regular expression
        match = re.search(r'c=(\d+.\d+),(\d+.\d+)', s)
        
        # get the name of the component
        name = elemh4.text
        
        # substitute for the weird words
        table = str.maketrans({
            '\u3000': ' ',
            ' ': '',
            '\t': ''
        })
        name = name.translate(table)
        
        # full-width to half-width
        name = name.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
        
        # get latitude and longitude
        try:
            latitude = match.group(1)
            longitude = match.group(2)
        except:
            pass
        
        # if there are same names of the components, delete one of such data
        if not name in name_list:
            name_list.append(name)
            
            # find the all stations by regular expression
            if re.compile(r".*駅.*").search(name):
                
                # append the name, latitude, and longitude
                name = name.replace("駅", "Station", 1)
                list.append(name)
                list.append(latitude)
                list.append(longitude)
                lists_all_stations.append(list)

In [51]:
# show the lists of station
for i in range(len(lists_all_stations)):
    print(lists_all_stations[i])

['北海道医療大学Station(学園都市線)', '43.231043706566965', '141.55002146817003']
['木古内Station(北海道新幹線)', '41.67768816717202', '140.43393772896002']
['新青森Station(北海道新幹線)', '40.82806721022598', '140.69343078656993']
['新函館北斗Station(北海道新幹線)', '41.904754407269024', '140.64861442158997']
['奥津軽いまべつStation(北海道新幹線)', '41.145095635678985', '140.51565228736']
['青森Station(津軽線)', '40.829336055837985', '140.73403843385995']
['青森Station(奥羽本線)', '40.829336055837985', '140.73403843385995']
['青森Station(青い森鉄道線)', '40.829336055837985', '140.73403843385995']
['新青森Station(奥羽本線)', '40.82806721022598', '140.69343078656993']
['新青森Station(東北新幹線)', '40.82806721022598', '140.69343078656993']
['東青森Station(青い森鉄道線)', '40.810532810086016', '140.78331299672004']
['岩手飯岡Station(東北本線)', '39.65375264701302', '141.14666475389004']
['岩手上郷Station(釜石線)', '39.28445916393602', '141.58604389572997']
['岩手船越Station(リアス線)', '39.431432992202986', '141.97401915301998']
['岩手川口Station(いわて銀河鉄道線)', '39.91884810316', '141.19929356587']
['岩手二日町Station

In [66]:
# scrape the name, latitude, and longitude of IC in MapFan

# define the search word and output list
search_word = "IC"

lists_IC = []
name_list = []

# loop for each area
for area in areas:
    
    # go to the MapFan site
    driver.get('https://mapfan.com/map/spots/search?c=35.681195934673795,139.7672311841094,12&s=std,pc,ja&p=none')
    sleep(1)        

    # search the search bar
    # enter the search word
    # get the result of the search
    search_input = driver.find_element_by_xpath('/html/body/mf-root/div/mat-sidenav-container/mat-sidenav-content/mf-map-page-map/mat-sidenav-container/mat-sidenav-content/div/div[1]/mf-map-page-search/div/form/mat-toolbar/mat-toolbar-row/mf-suggest-field/mat-form-field/div/div[1]/div[3]/input')
    search_input.send_keys("{} {}".format(search_word, area))
    search_input.send_keys(Keys.ENTER)


    count = 1
    
    # loop for name list
    for elemh4 in driver.find_elements_by_xpath('//h4'):
        elem = elemh4.find_element_by_xpath("../../..")
        
        #use p_text to get rid of the unintended words
        p_text = ""
        if count < len(driver.find_elements_by_xpath('//h4')):
            elem_aboves = elemh4.find_elements_by_xpath("../p")
            try: 
                p_text = elem_aboves[1].text
                count += 1
            except:
                pass
        
        # click one of the componenets in the list
        elem.click()
        sleep(1)
        
        # get the url of the component
        s = driver.current_url
        list = []
        
        # search latitude and longitude by regular expression
        match = re.search(r'c=(\d+.\d+),(\d+.\d+)', s)
        
        # get the name of the component
        name = elemh4.text
        
        # substitute for the weird words
        table = str.maketrans({
            '\u3000': ' ',
            ' ': '',
            '\t': ''
        })
        name = name.translate(table)
        
        # full-width to half-width
        name = name.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
        
        # get latitude and longitude
        try:
            latitude = match.group(1)
            longitude = match.group(2)
        except:
            pass
        
        # if there are same names of the components, delete one of such data
        if not name in name_list:
            name_list.append(name)
            
            # find the IKEA by regular expression
            if re.compile(r".*道.*").search(p_text) or p_text == ["交差点"]:
                
                # append the name, latitude, and longitude
                name2 = name.replace("IC", "Inter Change ", 1)
                list.append(name2)
                list.append(latitude)
                list.append(longitude)
                lists_IC.append(list)

In [67]:
# show the lists of IC
for i in range(len(lists_IC)):
    print(lists_IC[i])

['北海道銀行北海道大学病院', '43.07728752520299', '141.34441739794']
['北海道銀行左股通', '43.066327311956', '141.27384297747994']
['北海道銀行サンピアザ', '43.03758295149399', '141.47264368904007']
['北海道銀行JR恵庭駅', '42.88269231262302', '141.5867289135']
['青森Inter Change (青森自動車道)【青森東方面】', '40.79801092737401', '140.67611969400002']
['青森Inter Change (青森自動車道)【盛岡方面】', '40.79359418656401', '140.67284559850998']
['青森中央Inter Change (青森自動車道)【出口】', '40.796312406251985', '140.74759971927006']
['青森中央Inter Change (青森自動車道)【入口】', '40.79632523398803', '140.74746723052']
['青森Inter Change (東北自動車道)【青森東方面】', '40.79801092737401', '140.67611969400002']
['青森東Inter Change (青森自動車道)【出口(下り)】', '40.830135057088', '140.84011483829']
['青森東Inter Change (青森自動車道)【入口(上り)】', '40.830058900919', '140.84016121317995']
['青森Inter Change (東北自動車道)【盛岡方面】', '40.79359418656401', '140.67284559850998']
['青森Inter Change (東北自動車道)【入口(上り)】', '40.81226539091696', '140.68683647102']
['青森Inter Change (東北自動車道)【出口(下り)】', '40.81194716152001', '140.68627191382006']
['宮城川崎I

In [ ]:
# terminate the driver
driver.close()

In [70]:
def write_elem(lists, filename):
    filename += ".csv"
    filename = "CSV/" + filename
    for i in range(len(lists)):    
        with open(filename, "a", encoding="UTF-8") as f:
            f.write("{}, {}, {}\n".format(lists[i][0], lists[i][1], lists[i][2]))

In [71]:
write_elem(lists_IKEA, "IKEA")
write_elem(lists_COSTCO, "COSTCO")
write_elem(lists_LALAPORT, "LALAPORT")
write_elem(lists_NITORI, "NITORI")
write_elem(lists_DECOHOME, "DECOHOME")
write_elem(lists_station, "station")
write_elem(lists_all_stations, "all_stations")
write_elem(lists_IC, "IC")